In [1]:
%reset -fs
import pandas as pd
import seaborn as sns
import numpy as np
import requests
import time, os
from bs4 import BeautifulSoup
import datetime
import re
from datetime import date
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 500)

In [2]:
chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = chromedriver

In [3]:
driver = webdriver.Chrome(chromedriver)

In [4]:
bet_search =  'https://www.bettingpros.com/nba/picks/prop-bets/bet/points/'
query = '?date=' + datetime.date(2021, 10, 19).strftime('%Y-%m-%d')
bet_query = bet_search + query

In [5]:
base = datetime.datetime.today()
delta = base - datetime.datetime(2021, 10, 19)
date_list = [base - datetime.timedelta(days=x) for x in range(delta.days+1)]
date_list = [d.strftime('%Y-%m-%d') for d in date_list]

In [6]:
bet_query = bet_search + '?date=' + '2021-12-20'
driver.get(bet_query)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
# Attempt # 1 at getting all the pages of the JS table for each date's URL. Made decent progress here fizzled out in the while statement.
def get_data(date):
    data_str = []
    data = []
    li = []
    final_df = pd.DataFrame()
    bet_query = bet_search + '?date=' + date
    driver.get(bet_query)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    date = soup.find('h1').text
    date = date.split(" ")[5]
    date.strip(')')
    date = date.replace('(',"").replace(')',"")
    house_accuracy = soup.find('div', class_='best-props__accuracy').text
    table_data = table.tbody.find_all('tr')
    for tr in table_data:
        data_str.append(tr.text.replace('\n', ' ').strip())
    #for i in data_str:
        #li = list(i.split('            '))
        #data.append(li)
    while True:
        try:
            element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
            action = ActionChains(driver)
            action.move_to_element(element).click().perform()
             
            time.sleep(3)
            table_2 = soup.find('table')
            table_2_data = table_2.tbody.find_all('tr')
            for tr in table_2_data:
                data_str.append(tr.text.replace('\n', ' ').strip()) 
        except "inactive" in element.get_attribute("class"):
            break

In [ ]:
# The working get_data fxn. Used for MVP data.
def get_data(date):
    data_str = []
    data = []
    li = []
    final_df = pd.DataFrame()
    bet_query = bet_search + '?date=' + date
    driver.get(bet_query)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    date = soup.find('h1').text
    date = date.split(" ")[5]
    date.strip(')')
    date = date.replace('(',"").replace(')',"")
    house_accuracy = soup.find('div', class_='best-props__accuracy').text
    table_data = table.tbody.find_all('tr')
    for tr in table_data:
        data_str.append(tr.text.replace('\n', ' ').strip())
    for i in data_str:
        li = list(i.split('            '))
        data.append(li)
    for i in data:
        if len(i) == 5:
            i.append('TBD')
            i.append(date)
            i.append(house_accuracy)
        else:
            i.append(date)
            i.append(house_accuracy)
    final_df = final_df.append(pd.DataFrame(data))
    return final_df


In [ ]:
  # This cell is a bunch of miscellaneous code
   while True:
    try:
        driver.find_element_by_xpath('//*[@id="props-app"]/div/div/div[4]/div/div/a[2]').click()
        time.sleep(3)
    except NoSuchElementException:
        break
    if len(driver.find_elements_by_class_name('table__pagination')) > 0:
        element = driver.find_element_by_class_name('table_pagination')
        action = ActionChains(driver)
        action.move_to_element(element).click().perform()
        time.sleep(3)
        driver.execute_script('window.scrollTo(0, 0);')
        table_2 = soup.find('table')
        table_2_data = table_2.tbody.find_all('tr')
        for tr in table_2_data:
            data_str.append(tr.text.replace('\n', ' ').strip())
        if len(driver.find_elements_by_class_name('table__pagination')) > 0:
            element = driver.find_element_by_class_name('table_pagination')
            action = ActionChains(driver)
            action.move_to_element(element).click().perform()
            time.sleep(3)
            driver.execute_script('window.scrollTo(0, 0);')
            table_3 = soup.find('table')
            table_3_data = table_3.tbody.find_all('tr')
            for tr in table_3_data:
                data_str.append(tr.text.replace('\n', ' ').strip())
            if len(driver.find_elements_by_class_name('table__pagination')) > 0:
                element = driver.find_element_by_class_name('table_pagination')
                action = ActionChains(driver)
                action.move_to_element(element).click().perform()
                time.sleep(3)
                driver.execute_script('window.scrollTo(0, 0);')
                table_4 = soup.find('table')
                table_4_data = table_4.tbody.find_all('tr')
                for tr in table_4_data:
                    data_str.append(tr.text.replace('\n', ' ').strip())
    for i in data_str:
        li = list(i.split('            '))
        data.append(li)
    for i in data:
        if len(i) == 5:
            i.append('TBD')
            i.append(date)
            i.append(season_to_date_accuracy)
            i.append(seven_day_accuracy)
        else:
            i.append(date)
            i.append(season_to_date_accuracy)
            i.append(seven_day_accuracy)
    final_df = final_df.append(pd.DataFrame(data))
    return final_df


In [ ]:
final_df = get_data('2021-12-20')

In [ ]:
# Single date output for MVP get_data fxn
final_df

In [ ]:
#Janky way to handle the two days so far when there are no games and therefore no JS table loads
final_df = pd.DataFrame()
for i in date_list:
    if i == '2021-12-24':
        continue
    if i == '2021-11-25':
        continue
    
    final_df = final_df.append(get_data(i))

In [ ]:
# Raw output all dates
final_df

In [ ]:
df = final_df.copy()
df

In [ ]:
#Clean up below
df.columns=['Name', 'Team', 'Opp', 'Type', 'Line', 'Result', 'Date', 'Accuracy']
df

In [ ]:
player_name = df.Name.str.split(" ", expand = True)
player_name
df['Name'] = player_name[2] + ' ' + player_name[4]

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df

In [ ]:
team = df.Team.str.split(" ", expand = True)
team

In [ ]:
df['Team'] = team[0]
df['Position'] = team[2]
df

In [ ]:
opp = df.Opp.str.split(" ", expand = True)
opp

In [ ]:
df.Opp = opp[1]
df['Home/Away'] = opp[0]
df

In [ ]:
bet_info = df.Line.str.split(" ", expand = True)
bet_info

In [ ]:
df['Line'] = bet_info[7]
df['Projection'] = bet_info[15]
df['Diff'] = bet_info[9]
df['Moneyline'] = bet_info[13]
df

In [ ]:
df['Pick'] = bet_info[29]
df

In [ ]:
df = df[['Date', 'Name', 'Team', 'Position', 'Opp','Home/Away', 'Line', 'Moneyline', 'Projection', 'Diff', 'Pick', 'Result', 'Accuracy']]
df

In [ ]:
picks = df.Pick
picks
picks = [list(i) for i in picks]

In [ ]:
over_under = [i[0] for i in picks]
over_under

In [ ]:
df['pick_over_under'] = over_under
df

In [ ]:
df['Pick'] = df['Pick'].map(lambda x: x.lstrip('ou'))
df

In [ ]:
accuracies = df.Accuracy.str.split(" ", expand = True)
accuracies

In [ ]:
df['seven_day_accuracy'] = accuracies[8]

In [ ]:
df.seven_day_accuracy

In [ ]:
df.seven_day_accuracy.value_counts()

In [ ]:
df['seven_day_accuracy'] = df['seven_day_accuracy'].map(lambda x: x.lstrip('(').rstrip('%)'))

In [ ]:
df.seven_day_accuracy = df.seven_day_accuracy.replace('', 50, regex=True)

In [ ]:
df['seven_day_accuracy'] = pd.to_numeric(df['seven_day_accuracy'])

In [ ]:
df.dtypes

In [ ]:
df['seven_day_accuracy'] = df['seven_day_accuracy'].map(lambda x: x/100)

In [ ]:
df.seven_day_accuracy.value_counts()

In [ ]:
df.drop(columns=['Accuracy'], inplace=True)
df

In [ ]:
# Final form betting df, additional cleaning and saving to csv below
sorted_df = df.sort_values(by=['Name', 'Date'])
sorted_df.reset_index(inplace=True, drop=True)
sorted_df

In [ ]:
sorted_df['Line'] = pd.to_numeric(sorted_df.Line)

In [ ]:
sorted_df = sorted_df.replace('EVEN', '+100')

In [ ]:
sorted_df = sorted_df.replace('TBD', np.nan)

In [ ]:
sorted_df['Moneyline'] = pd.to_numeric(sorted_df.Moneyline)

In [ ]:
sorted_df['Projection'] = pd.to_numeric(sorted_df.Projection)
sorted_df['Diff'] = pd.to_numeric(sorted_df.Diff)
sorted_df['Result'] = pd.to_numeric(sorted_df.Result)
sorted_df['Pick'] = pd.to_numeric(sorted_df.Pick)

In [ ]:
sorted_df

In [ ]:
sorted_df.dtypes

In [ ]:
# Determine if website pick was a win or loss (for fun, potential spinoff angle).
conditions = [
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['pick_over_under'] == 'u')),
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['pick_over_under'] == 'o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['pick_over_under'] =='o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['pick_over_under'] == 'u'))
    ]

# create a list of the values we want to assign for each condition
values = ['1', '0', '1', '0']

# create a new column and use np.select to assign values to it using our lists as arguments
sorted_df['successful_pick'] = np.select(conditions, values)

In [ ]:
sorted_df

In [ ]:
sorted_df.to_csv('betting_data.csv')

In [ ]:
#SCRATCH
element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
#element = driver.find_element_by_xpath('//a[contains(@aria-label, "Display Next Page of results")]')
action = ActionChains(driver)
action.move_to_element(element).click().perform()


In [ ]:
#SCRATCH
page_progress = driver.find_element_by_xpath('//div[contains(@class, "table__pagination")]').text.strip("«").strip("»").strip()


In [ ]:
#SCRATCH
page_progress

In [ ]:
#SCRATCH
if int(page_progress[5]) == int(page_progress[10]):
    break
else:
    element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
    action = ActionChains(driver)
    action.move_to_element(element).click().perform()

In [7]:
#run this initially; then don't run again if you just want to keep adding to it
final_df = pd.DataFrame()

In [8]:
driver = webdriver.Chrome(chromedriver)

In [9]:
def get_site_rb(date):
    bet_query = bet_search + '?date=' + date
    driver.get(bet_query)
    time.sleep(5)
    element = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
    action = ActionChains(driver)
    action.move_to_element(element).click().perform()


In [18]:
def get_data_rb():
    data_str = []
    data = []
    li = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    date = soup.find('h1').text
    date = date.split(" ")[5]
    date.strip(')')
    date = date.replace('(',"").replace(')',"")
    house_accuracy = soup.find('div', class_='best-props__accuracy').text
    table_data = table.tbody.find_all('tr')
    for tr in table_data:
        data_str.append(tr.text.replace('\n', ' ').strip())
    for i in data_str:
        li = list(i.split('            '))
        data.append(li)
    for i in data:
        if len(i) == 5:
            i.append('TBD')
            i.append(date)
            i.append(house_accuracy)
        else:
            i.append(date)
            i.append(house_accuracy)
    global final_df
    final_df = final_df.append(pd.DataFrame(data))


    page_progress = driver.find_element_by_xpath('//div[contains(@class, "table__pagination")]').text.strip("«").strip("»").strip()
    if int(page_progress[5]) == int(page_progress[10]):
        return final_df
    else:
        element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
        action = ActionChains(driver)
        action.move_to_element(element).click().perform()
        get_data_rb()


In [10]:
#run this and then close the cookies pop-up!
get_site_rb('2021-12-20')

In [24]:
#then run this
for i in date_list:
    get_data_rb()

KeyboardInterrupt: 

In [ ]:
driver.close()